In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re

import tensorflow as tf

from tensorflow import keras
from tensorflow.config import set_visible_devices, get_visible_devices

from tqdm.auto import tqdm

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

#comment this @Nikita
from interface.params import *

/Users/annoskriver/.pyenv/versions/3.10.6/envs/blackjack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [ ]:
# Fill this out @Nikita

# TRAIN_DIR_IMGS  = '/Users/annoskriver/.data/blackjack/original_images/train/images'
# TRAIN_DIR_LABELS = '/Users/annoskriver/.data/blackjack/original_images/train/labels'

# VALID_DIR_IMGS = '/Users/annoskriver/.data/blackjack/original_images/valid/images'
# VALID_DIR_LABELS = '/Users/annoskriver/.data/blackjack/original_images/valid/labels'

# TEST_DIR_IMGS = '/Users/annoskriver/.data/blackjack/original_images/test/images'
# TEST_DIR_LABELS = '/Users/annoskriver/.data/blackjack/original_images/test/labels'

# MODEL_PATH = '/Users/annoskriver/.data/blackjack/models'

In [3]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCH = 128
GLOBAL_CLIPNORM = 10.0      
IMAGE_SIZE = 416
USE_GPU = True

In [4]:
#delete GPU from visible devices, if not used
if not USE_GPU:
    set_visible_devices([], 'GPU')

#look at visible devices
get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# Class mapping
class_ids = ['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [6]:
def get_labels(labels_dir: str, img_size: int = 416) -> (list, list):
    file_paths = []
    classes = []
    boxes = []
    # Get labels: class ID and x, y, width, height (bounding boxes)
    for root, dirs, files in os.walk(labels_dir):
        for file in files:
            # Get path for each label .txt file
            label_path = os.path.join(labels_dir, os.path.splitext(file)[0] + ".txt")
            file_paths.append(label_path)

        # Sort paths by picture name
        pattern = r"(\d+)_jpg\.rf"
        file_paths.sort(key=lambda x: int(re.findall(pattern, x)[0]))

        for file in file_paths:
            # Open label .txt file
            with open (file, "r") as label_file:
                label_lines = label_file.readlines()

                img_classes = []
                img_boxes = []

                # Parse label lines
                for line in label_lines:
                    parts = line.split()
                    single_class = int(parts[0])  # First value is the class label
                    img_classes.append(single_class)

                    single_box = [float(x) * img_size for x in parts[1:]]  # Rest are the boxes (x,y and height,width)
                    img_boxes.append(single_box)

                classes.append(img_classes)
                boxes.append(img_boxes)
    return classes, boxes

In [7]:
train_classes, train_boxes = get_labels(TRAIN_DIR_LABELS)
valid_classes, valid_boxes = get_labels(VALID_DIR_LABELS)
test_classes, test_boxes = get_labels(TEST_DIR_LABELS)

In [8]:
def get_imgs(imgs_dir: str) -> (list):
    paths = []
    for root, dirs, files in os.walk(imgs_dir):
        for file in files:
            img_path = os.path.join(imgs_dir, os.path.splitext(file)[0] + ".jpg")
            paths.append(img_path)
    paths.sort()
    return paths

In [9]:
train_img_paths = get_imgs(TRAIN_DIR_IMGS)
valid_img_paths = get_imgs(VALID_DIR_IMGS)
test_img_paths = get_imgs(TEST_DIR_IMGS)

In [10]:
len(train_img_paths), len(valid_img_paths)

(14000, 4000)

In [11]:
# Check correct size
assert len(train_classes) == len(train_boxes) == len(train_img_paths)
assert len(valid_classes) == len(valid_boxes) == len(valid_img_paths)
assert len(test_classes) == len(test_boxes) == len(test_img_paths)

In [12]:
# Creating data, after converting lists to ragged tensors
# Train dataset:
train_classes_tf = tf.ragged.constant(train_classes)
train_boxes_tf = tf.ragged.constant(train_boxes)
train_paths_tf = tf.ragged.constant(train_img_paths)

train_data = tf.data.Dataset.from_tensor_slices((train_paths_tf, train_classes_tf, train_boxes_tf))

# Validation dataset:
valid_classes_tf = tf.ragged.constant(valid_classes)
valid_boxes_tf = tf.ragged.constant(valid_boxes)
valid_paths_tf = tf.ragged.constant(valid_img_paths)

valid_data = tf.data.Dataset.from_tensor_slices((valid_paths_tf, valid_classes_tf, valid_boxes_tf))

# Test dataset:
test_classes_tf = tf.ragged.constant(test_classes)
test_boxes_tf = tf.ragged.constant(test_boxes)
test_paths_tf = tf.ragged.constant(test_img_paths)

test_data = tf.data.Dataset.from_tensor_slices((test_paths_tf, test_classes_tf, test_boxes_tf))

2023-09-04 16:59:17.923482: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-09-04 16:59:17.923501: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-09-04 16:59:17.923504: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-09-04 16:59:17.923923: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-04 16:59:17.924234: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    return image # Loading data functions


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": image, "bounding_boxes": bounding_boxes}

In [14]:
# Data augmentater for the training dataset
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="center_xywh"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="center_xywh"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(416, 416), scale_factor=(0.75, 1.3), bounding_box_format="center_xywh"
        ),
    ]
)

In [15]:
# Creating training dataset
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

In [16]:
# Data resizer for the validation dataset
resizing = keras_cv.layers.JitteredResize(
    target_size=(416, 416),
    scale_factor=(0.75, 1.3),
    bounding_box_format="center_xywh",
)

In [17]:
# Creating validation dataset
val_ds = valid_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [18]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )

In [19]:
#visualize_dataset(train_ds, bounding_box_format="center_xywh", value_range=(0,255), rows=2, cols=2)

# MODEL CREATION (YOLOv8)

In [20]:
# YOLOv8 backbone
backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")  # yolov8 backbone with coco weights

In [21]:
# Build the model
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="center_xywh",
    backbone=backbone,
    fpn_depth=1, # Feature Pyramid Network
)

In [22]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE, global_clipnorm=GLOBAL_CLIPNORM,)

yolo.compile(optimizer=optimizer, classification_loss = "binary_crossentropy" , box_loss="ciou")

In [23]:
# Model evaluation with COCO Metric Callback
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, data, save_path):
        super().__init__()
        self.data = data
        self.metrics = keras_cv.metrics.BoxCOCOMetrics(
            bounding_box_format="center_xywh",
            evaluate_freq=1e9,
        )

        self.save_path = save_path
        self.best_map = -1.0

    def on_epoch_end(self, epoch, logs):
        self.metrics.reset_state()
        for batch in self.data:
            images = batch["images"]
            bounding_boxes = batch["bounding_boxes"]

            # Extract "boxes" and "classes" from bounding_boxes
            classes = bounding_boxes["classes"]
            boxes = bounding_boxes["boxes"]

            y_pred = self.model.predict(images, verbose=0)

            # Convert classes and bounding_boxes to a dictionary
            y_true = {"classes": classes, "boxes": boxes}

            self.metrics.update_state(y_true, y_pred)

        metrics = self.metrics.result(force=True)
        logs.update(metrics)

        current_map = metrics["MaP"]
        if current_map > self.best_map:
            self.best_map = current_map
            self.model.save_weights(self.save_path)  # Save the model when mAP improves

        return logs

In [24]:
# Model training
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    callbacks=[EvaluateCOCOMetricsCallback(val_ds, os.path.join(MODEL_PATH, "best_coco_model_autosave.keras")), 
               tqdm.keras.TQDMCallback(verbose=1)],
    verbose=1,
    batch_size=BATCH_SIZE
)

2023-09-04 16:59:24.494451: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-04 16:59:24.807249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-04 16:59:30.657289: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
# Save the fitted model under name

model_save_name = "final_epoch_model.keras"

yolo.save_weights(os.path.join(MODEL_PATH, model_save_name))

In [ ]:
# Loading the saved model...

# Define model architecture
def create_custom_model():
    backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")
    yolo = keras_cv.models.YOLOV8Detector(
        num_classes=len(class_mapping),
        bounding_box_format="center_xywh",
        backbone=backbone,
        fpn_depth=1,
    )
    return yolo

# Create the model
reconstructed_yolo = create_custom_model()

# Load the model weights
reconstructed_yolo.load_weights(os.path.join(MODEL_PATH, "best_coco_model_autosave.keras"))

In [ ]:
# Load and preprocess your input image
image_path = "/Users/annoskriver/.data/blackjack/original_images/train/images/000090528_jpg.rf.d50e89610e5c97c61632c290692f3e75.jpg"
input_image = load_image(image_path)
input_image = tf.expand_dims(input_image, axis=0)  # Add a batch dimension

# Make predictions
predictions = reconstructed_yolo.predict(input_image)

1/1 [==============================] - 2s 2s/step


In [ ]:
predictions["classes"]

In [ ]:
predictions["images"]

In [ ]:
# Visualize predictions
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )